In [1]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy

In [2]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"

In [3]:
from moscot.problems.time._lineage import TemporalProblem

In [4]:
Path="/home/mgander/mouse_atlas/data"
ts=['E3.5', 'E4.5', 'E5.25', 'E5.5', 'E6.25', 'E6.5', 'E6.75', 'E7.0', 'E7.25', 'E7.5', 'E7.75', 'E8.0', 'E8.25', 'E8.5a', 'E8.5b', 'E9.5', 'E10.5', 'E11.5', 'E12.5', 'E13.5']

In [ ]:
for i in range(13):
    ts0=ts[i]
    ts1=ts[i+1]
    print(f'{ts0}_{ts1}')

    adata=sc.read(f"{Path}/Comb_anndatas/adata_{ts0}_{ts1}.h5ad")
    del adata.raw
    adata.obs['day']=adata.obs['day'].astype('category')
    day0,day1=sorted(set(adata.obs['day']))
    inds0=list(adata[adata.obs['day']==day0].obs.index)
    inds1=list(adata[adata.obs['day']==day1].obs.index)
    
    
    eps=0.005
    tau1s=[0.99,0.98,0.99,0.92,0.95,0.9,0.3,0.65,0.9,0.82,0.93,0.97,0.96]
    lam1s=[t*eps/(1-t) for t in tau1s]
    lam2=100
    lam1=lam1s[i]
    tau1=lam1/(lam1+eps)
    tau2=lam2/(lam2+eps)
    
    
    tp=TemporalProblem(adata)
    tp.score_genes_for_marginals(gene_set_proliferation='mouse',  gene_set_apoptosis='mouse')
    tp = tp.prepare('day', joint_attr=f'X_pcaS')
    result=tp.solve(batch_size=None, epsilon=eps, tau_a=tau1, tau_b=tau2, scale_cost="median")
    
    tmap=result[(day0, day1)].solution.transport_matrix
    T=pd.DataFrame(data=tmap, index=inds0, columns=inds1)
    T=T/T.sum()
    T.to_pickle(f'{Path}/moscot_maps/Ts/Transitions_{ts0}_{ts1}.pkl')